In [ ]:
import pandas as pd
import requests
from functools import reduce

def EarningsData(symbol):
    nos_list=[]
    for i in range(24):
        nos_list.append(str(-23+i))
    rel_periods = ','.join(nos_list)
    
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-meta-data"
    querystring = {"symbol":symbol}
    headers = {
        "X-RapidAPI-Key": "48dc2a3246msh40981dcaf875956p13aa7bjsn0b358ff118ae",
        "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    id_data = response.json()

    eps_id = id_data['data']['id']
    
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-earnings"
    querystring = {"ticker_ids":eps_id,"period_type":"quarterly","relative_periods":rel_periods,"estimates_data_items":"eps_gaap_actual,eps_gaap_consensus_mean,eps_normalized_actual,eps_normalized_consensus_mean,revenue_actual,revenue_consensus_mean","revisions_data_items":"eps_normalized_actual,eps_normalized_consensus_mean,revenue_consensus_mean"}
    headers = {
        "X-RapidAPI-Key": "48dc2a3246msh40981dcaf875956p13aa7bjsn0b358ff118ae",
        "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    metrics = data['estimates'][eps_id].keys()

    df_outer_list = []
    for nos in nos_list:
        df_inner_list = []
        for metric in metrics:
            try:
                dict = data['estimates'][eps_id][metric][nos][0]['period']
            except KeyError:
                continue
            dict_new = {key: dict[key] for key in ['fiscalquarter','fiscalyear', 'calendarquarter', 'calendaryear']}
            dict_new[metric] = data['estimates'][eps_id][metric][nos][0]['dataitemvalue']
            df_inner_list.append(pd.DataFrame(dict_new, index=[0])) 
        df_outer_list.append(df_inner_list)

    df_new_list = []
    for df_list in df_outer_list:
        df_new_list.append(reduce(lambda x, y: pd.merge(x, y, how='outer', on = ['fiscalquarter','fiscalyear', 'calendarquarter', 'calendaryear']), df_list))

    earnings_df = pd.concat(df_new_list)
    earnings_df.reset_index(inplace=True)
    earnings_df = earnings_df.drop(columns=['index'])

    symbol_list = []
    for i in range(len(earnings_df.index)):
        symbol_list.append(symbol)

    earnings_df.insert(0, 'symbol', symbol_list)
    return earnings_df